In [1]:
# ____________ _____ _____ _____   _____ _____ _________________   
#|  ___| ___ \_   _|_   _/  ___| /  __ \  _  |  ___|  ___|  ___|  
#| |_  | |_/ / | |   | | \ `--.  | /  \/ | | | |__ | |_  | |_ ___ 
#|  _| |    /  | |   | |  `--. \ | |   | | | |  __||  _| |  _/ __|
#| |   | |\ \ _| |_ _| |_/\__/ / | \__/\ \_/ / |___| |   | | \__ \
#\_|   \_| \_|\___/ \___/\____/   \____/\___/\____/\_|   \_| |___/
                                                                 
## This is a test environment for the FRIIS COEFFICIENT calculations
## This will hopefully become a part of some concatenation class in the data pipeline
## This will combine telescope data with drone data, so both will be needed as input
## Additionally this must pull and interpolate the Bicolog Beam model

In [2]:
## Actual Debugging Work:
    # 1) Make the FRIIS COEFFICIENT Array in class format:
    

In [ ]:
    def Generate_FRIIS_Coeffs(self,t_cut=False,t_bounds=[0,-1]):
        ## Friis Equation Coefficients will be coeff of proportionality between Receiver Beam D and Pr/Pt ratio
        ## This will be initialized as a 4d array: (Receiver, Polarization, Time, Freq)
        

In [ ]:
## Calculating "FRIIS COEFFICIENT" Array, which is the proportionality between Receiver Beam and Pr/Pt ratio ##
    # For Autocorrelation data, this will have to be 4 dimensional: (Receiver, Polarization, Time, Freq)
    # This array can be multiplied elementwise by the power ratio (Pr/Pt) which will require CALIBRATED DATA.
    # The product of those is the Receiver Beam: D_r([theta,phi],receiver,polarization,freq)[time]
    # For each receiver, we will need to measure the position and pointing relative to our coordinate origin!
    
## Calculate Scalar Factor Array: (4*pi*R/lambda)**2
    # The radius R will be different for each receiver
    # The wavelength is a function of frequency
Wavelength_Array=(3.0e8/1.0e6)*np.outer(np.ones(len(time)),freq)**-1.0
R_Array=np.outer(data.coords_rpt[:,0],np.ones(len(freq)))
Sc_Factor=(4.0*np.pi*Wavelength_Array/R_Array)**2.0

## Calculate Efficiency Factor Array:  1/(e_t0*e_r0)
    # To actually calculate this, we will need efficiency measurements for each receiver polarization per freq.
    # We can include these later, if we collect these measurements, interpolate to frequencies in data
    # For now, it will all be ones (unity)!
Ecdt=1.0*np.ones(len(freq))        #conduction and dielectric (f)
Gammat=1.0*np.ones(len(freq))      #reflection coeff (f)
Eff_Trans=Ecdt*(1.0-(Gammat)**2.0) # e_t0 = total efficiency of transmitter (f)
Ecdr=1.0*np.ones(len(freq))
Gammar=1.0*np.ones(len(freq))
Eff_Rec=Ecdr*(1.0-(Gammar)**2.0)   # e_r0 = total efficiency of receiver (f)
Eff_Factor=np.outer(np.ones(len(time)),(Eff_Trans*Eff_Rec)**-1.0)

## Calculate PLF Factor: 1/mag(trans_pol_vec.rec_pol_vec)**2
## Define the pointing direction for the drone as a function of time, from rotating Nose direction:
drone_pol_pointing=np.array([RotMat(np.array([90.0,0.0,180.0]))@RotMat(ypr[m,:])@np.array([1,0,0]) for m in range(len(data.t_index))])
## Calculate the PLF (POLARIZATION LOSS FACTOR) for each receiver/polarization as fxn of time:
PLF_Factor=np.zeros((4,2,len(data.t_index)))
for j in range(4):
    for k in range(2):
        PLF_Factor[j,k,:]=np.array([np.dot(drone_pol_pointing,Array_Pols[j,k,:])**2.0])

## Calculate Transmitter Directivity Factor Array: 1/Dt(theta_t,phi_t)
    # Depends on receiver position as seen by drone transmitter as a function of time


## Initialize "FRIIS_COEFF_ARRAY" -- the output data product, which has same dims as data:
FRIIS_COEFF_ARRAY=Sc_Factor*Eff_Factor
